# RAG Testing

In [40]:
import langchain_community.llms
from langchain_ollama import ChatOllama
print(langchain_community.llms.__file__)
from langchain_community.embeddings import OllamaEmbeddings

/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/lib/python3.13/site-packages/langchain_community/llms/__init__.py


In [41]:
model = ChatOllama(model='gpt-oss', max_retry=5)
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

In [42]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [43]:
from typing import Iterable
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# docs should already have metadata attached. Document objects have a 'metadata' field that is just a dictionary
# (https://reference.langchain.com/python/langchain_core/documents/#langchain_core.documents.base.Document.metadata)
def insert_into_vector_store(docs: Iterable[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # chunk size (characters)
        chunk_overlap=100,  # chunk overlap (characters)
        add_start_index=True,  # track index in original document
    )
    all_splits = text_splitter.split_documents(docs)

    document_ids = vector_store.add_documents(documents=all_splits)

    print(f"Split docs into {len(all_splits)} sub-documents.")


In [44]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def remember(query: str):
    """Remember relevant past events during gameplay in order to inform the next step"""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [45]:
import jericho
print(jericho.__file__)
import sys
print(sys.path)

GAMES_DIR = "z-machine-games-master/jericho-game-suite"
game = 'zork1.z5'
env = jericho.FrotzEnv(f"{GAMES_DIR}/{game}")

/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/lib/python3.13/site-packages/jericho/__init__.py
['/usr/local/python/3.13.5/lib/python313.zip', '/usr/local/python/3.13.5/lib/python3.13', '/usr/local/python/3.13.5/lib/python3.13/lib-dynload', '', '/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/lib/python3.13/site-packages']


In [46]:
import time
def n_steps(turn_func, env, n=100):
    # Begin
    generate_times = []
    unique_rooms = set()
    unique_items = set()
    unique_hashes = set()
    retries = 0
    retries_per_score = []
    prev_score = 0

    for _ in range(n):

        # Turn
        start = time.time()
        done, info = turn_func()
        generate_times.append(time.time() - start)
        
        unique_rooms.add(env.get_player_location().name)
        unique_items.update([item.name for item in env.get_inventory()])
        unique_hashes.add(env.get_world_state_hash())
        
        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']

        if done:
            break

    return {
        'unique_rooms': len(unique_rooms),
        'unique_items': len(unique_items),
        'unique_hashes': len(unique_hashes),
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else f'{n}+',
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [ ]:
from langchain.agents import create_agent
from langchain_core.documents import Document
from ollama import ResponseError

def rag_agent():
    game_response, info = env.reset()

    done = False

    @tool(response_format="content")
    def do_game_action(action: str) -> str:
        """Perform an action in the active text adventure game and see the result"""
        """
        Args:
          action: game action string

        Returns:
          The game's response after performing the action
        """
        nonlocal done, game_response, info
        game_response, reward, done, info = env.step(action)
        if done:
            game_response += '\nYou have finished the game!'
        return game_response
    
    @tool(response_format="content")
    def view_possible_actions() -> str:
        """View a list of the actions that can be performed in the game's current state"""
        """
        Returns:
          String containg actions separated by commas
        """
        return ', '.join(env.get_valid_actions())        


    tools = [remember, do_game_action, view_possible_actions]
    system_prompt = (
        f"You are playing {game}, an interactive fiction game. You must analyze the scenario the game presents to you and choose an action that will make progress. Your goal is to finish the game\n"
        "You have access to a tool that allows you to remember past events that have occured in your current playthrough that are relevant to your situation. "
        "Use the tool to help you decide on the next action to take in-game "
    )
    agent = create_agent(model, tools, system_prompt=system_prompt)
    
    def agent_stream():
        nonlocal agent, game_response
        query = (
            "Think critically. Finish the game.\n"
            f"Here are relevant items from your past moves:\n{remember.invoke({'query':game_response})}\n"
            f"Here is your current scenario:\n{game_response}"
        )
        try:
            for event in agent.stream(
                {"messages": [{"role": "user", "content": query}]},
                stream_mode="values",
            ):
                last_message = event["messages"][-1]
                last_message.pretty_print()
                
                document = Document(
                    page_content=last_message.content, metadata={"move": info['moves']}
                )
                insert_into_vector_store([document])
                yield None
        except ResponseError:
            print('ResponseError occurred')
            
    cur_stream = None
    def turn():
        nonlocal cur_stream, done, info
        if cur_stream is None:
            cur_stream = agent_stream()
        try:
            next(cur_stream)
        except StopIteration:
            cur_stream = agent_stream()
        return done, info
    
    results = n_steps(turn, env)
    print(results)
    return results


In [48]:
rag_agent()

================================ Human Message =================================

Think critically. Finish the game.
Here are relevant items from yourpast moves:

Here is your current scenario:
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


Split docs into 1 sub-documents.
================================== Ai Message ==================================
Tool Calls:
  do_game_action (a4c54de8-0166-45cb-bcdf-95dbfcf0e3f9)
 Call ID: a4c54de8-0166-45cb-bcdf-95dbfcf0e3f9
  Args:
    action: open mailbox
Split docs into 0 sub-documents.
================================= Tool Message =================================
Name: do_game_action

Opening the small mailbox reveals a leaflet.


Split docs into 1 sub-documents.
================================== Ai Message =

{'unique_rooms': 7,
 'unique_items': 4,
 'unique_hashes': 17,
 'score': 10,
 'max_score': 350,
 'avg_retries': 36.0,
 'avg_generate_time': 11.819121100902557}